# Kentucky Water Quality Analysis  
## Keystone Project — Week 1

This notebook documents the initial exploration, inspection, and early cleaning of EPA Safe Drinking Water Act (SDWA) violations data for Kentucky public water systems.

---

##  Working Notes 

Below are common checks when working with and exploring a dataset. Not all code will be used for final analysis.


## 1. Data Loading

The first step of this Keystone Project involved loading three datasets:
- SDWA violations
- Kentucky public water systems
- Contaminant reference codes

These datasets will all be needed to see the drinking water quality issues in Kentucky.


In [22]:
from pathlib import Path
import pandas as pd

project_root = Path("..")
data_folder = project_root / "datasets" / "SDWA_ky"

violations_path = data_folder / "ky_violations_enforcement.csv"
systems_path = data_folder / "ky_pub_water_systems.csv"
codes_path = data_folder / "ref_contaminant_codes.csv"

violations = pd.read_csv(violations_path, low_memory=False)
systems = pd.read_csv(systems_path, low_memory=False)
codes = pd.read_csv(codes_path, low_memory=False)

violations.head()



,SUBMISSIONYEARQUARTER,PWSID,VIOLATION_ID,FACILITY_ID,COMPL_PER_BEGIN_DATE,COMPL_PER_END_DATE,NON_COMPL_PER_BEGIN_DATE,NON_COMPL_PER_END_DATE,PWS_DEACTIVATION_DATE,VIOLATION_CODE,...,RULE_FAMILY_CODE,VIOL_FIRST_REPORTED_DATE,VIOL_LAST_REPORTED_DATE,ENFORCEMENT_ID,ENFORCEMENT_DATE,ENFORCEMENT_ACTION_TYPE_CODE,ENF_ACTION_CATEGORY,ENF_ORIGINATOR_CODE,ENF_FIRST_REPORTED_DATE,ENF_LAST_REPORTED_DATE
0,2025Q3,CA4600082,1102003,NaN,08/01/2011,08/31/2011,08/01/2011,08/31/2011,NaN,44,...,120.0,06/26/2013,11/20/2015,1202005,10/26/2011,SFL,Formal,S,06/26/2013,11/20/2015
1,2025Q3,CA4600082,1102003,NaN,08/01/2011,08/31/2011,08/01/2011,08/31/2011,NaN,44,...,120.0,06/26/2013,11/20/2015,9502003,02/01/2012,SOX,Resolving,S,03/28/2018,02/16/2021
2,2025Q3,CA4600082,302001,NaN,10/01/2002,10/31/2002,10/01/2002,10/31/2002,NaN,21,...,110.0,07/10/2004,11/20/2015,1002004,11/30/2002,SOX,Resolving,S,06/04/2010,11/20/2015
3,2025Q3,FL3424693,20130003421,NaN,01/01/2012,12/31/2012,01/01/2012,12/31/2012,NaN,03,...,330.0,02/19/2013,04/03/2016,376208,02/21/2013,SOX,Resolving,S,09/24/2013,04/03/2016
4,2025Q3,FL3535116,20160005292,NaN,01/01/2016,NaN,01/01/2016,06/30/2016,NaN,52,...,350.0,08/29/2016,05/21/2019,467995,06/30/2016,SOX,Resolving,S,03/15/2017,05/21/2019


## 2. Initial Data Inspection

After loading the data, I inspected the dataset and looked at size, structure, data types, and missing values. 


In [23]:
violations.shape


(136596, 38)

In [24]:
violations.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136596 entries, 0 to 136595
Data columns (total 38 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   SUBMISSIONYEARQUARTER         136596 non-null  object 
 1   PWSID                         136596 non-null  object 
 2   VIOLATION_ID                  115934 non-null  object 
 3   FACILITY_ID                   17379 non-null   float64
 4   COMPL_PER_BEGIN_DATE          115934 non-null  object 
 5   COMPL_PER_END_DATE            105922 non-null  object 
 6   NON_COMPL_PER_BEGIN_DATE      115934 non-null  object 
 7   NON_COMPL_PER_END_DATE        115934 non-null  object 
 8   PWS_DEACTIVATION_DATE         47223 non-null   object 
 9   VIOLATION_CODE                115934 non-null  object 
 10  VIOLATION_CATEGORY_CODE       115934 non-null  object 
 11  IS_HEALTH_BASED_IND           115934 non-null  object 
 12  CONTAMINANT_CODE              115903 non-nul

## 3. Exploring Key Variables

I explored some key variables to see the types of violations including violation categories, health-based indicators, and severity measures.


In [25]:
violations["VIOLATION_CATEGORY_CODE"].value_counts(dropna=False).head(15)


VIOLATION_CATEGORY_CODE
MR       94371
NaN      20662
MCL      11306
Other     5892
TT        3456
MON        810
RPT         97
MRDL         2
Name: count, dtype: int64

In [26]:
violations["IS_HEALTH_BASED_IND"].value_counts(dropna=False)





IS_HEALTH_BASED_IND
N      101170
NaN     20662
Y       14764
Name: count, dtype: int64

In [27]:
violations["SEVERITY_IND_CNT"].describe()


count    5467.000000
mean        1.613316
std         0.910619
min         1.000000
25%         1.000000
50%         1.000000
75%         2.000000
max        12.000000
Name: SEVERITY_IND_CNT, dtype: float64

## 4. Missing Data Assessment

SDWA datasets contain many fields that are only applicable to certain violation types. I looked at missing data patterns to see which columns may not be usable for analysis.


In [28]:
missing_counts = violations.isna().sum().sort_values(ascending=False)
missing_counts.head(15)


CORRECTIVE_ACTION_ID       136596
SAMPLE_RESULT_ID           136459
SEVERITY_IND_CNT           131129
UNIT_OF_MEASURE            127596
FACILITY_ID                119217
STATE_MCL                  110234
VIOL_MEASURE               109158
PWS_DEACTIVATION_DATE       89373
IS_MAJOR_VIOL_IND           54652
COMPL_PER_END_DATE          30674
ENF_ACTION_CATEGORY         22610
FEDERAL_MCL                 21803
CALCULATED_RTC_DATE         20733
CONTAMINANT_CODE            20693
VIOL_LAST_REPORTED_DATE     20681
dtype: int64

In [29]:
missing_percent = (violations.isna().mean() * 100).sort_values(ascending=False)
missing_percent.head(15)


CORRECTIVE_ACTION_ID       100.000000
SAMPLE_RESULT_ID            99.899704
SEVERITY_IND_CNT            95.997687
UNIT_OF_MEASURE             93.411227
FACILITY_ID                 87.277080
STATE_MCL                   80.700753
VIOL_MEASURE                79.913028
PWS_DEACTIVATION_DATE       65.428709
IS_MAJOR_VIOL_IND           40.009956
COMPL_PER_END_DATE          22.456002
ENF_ACTION_CATEGORY         16.552461
FEDERAL_MCL                 15.961668
CALCULATED_RTC_DATE         15.178336
CONTAMINANT_CODE            15.149053
VIOL_LAST_REPORTED_DATE     15.140268
dtype: float64

## 5. Data Cleaning and Wrangling

Basic cleaning and wrangling steps including checking for duplicates, filtering, sorting records, renaming columns, and converting dates to datetime format.


In [30]:
violations.duplicated().sum()


np.int64(0)

In [31]:
violations = violations.drop_duplicates()
violations.duplicated().sum()


np.int64(0)

In [32]:
health_violations = violations[violations["IS_HEALTH_BASED_IND"] == "Y"]
health_violations.shape


(14764, 38)

In [33]:
violations.sort_values("NON_COMPL_PER_BEGIN_DATE").head()


,SUBMISSIONYEARQUARTER,PWSID,VIOLATION_ID,FACILITY_ID,COMPL_PER_BEGIN_DATE,COMPL_PER_END_DATE,NON_COMPL_PER_BEGIN_DATE,NON_COMPL_PER_END_DATE,PWS_DEACTIVATION_DATE,VIOLATION_CODE,...,RULE_FAMILY_CODE,VIOL_FIRST_REPORTED_DATE,VIOL_LAST_REPORTED_DATE,ENFORCEMENT_ID,ENFORCEMENT_DATE,ENFORCEMENT_ACTION_TYPE_CODE,ENF_ACTION_CATEGORY,ENF_ORIGINATOR_CODE,ENF_FIRST_REPORTED_DATE,ENF_LAST_REPORTED_DATE
72112,2025Q3,KY0392416,988881,NaN,01/01/1978,12/31/1980,01/01/1978,12/31/1980,10/01/1981,03,...,330.0,03/27/1991,03/27/1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64674,2025Q3,KY0672475,1017281,NaN,01/01/1978,12/31/1980,01/01/1978,12/31/1980,08/01/2003,03,...,330.0,07/10/1990,03/27/1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100384,2025Q3,KY1022558,1049681,NaN,01/01/1978,12/31/1980,01/01/1978,12/31/1980,07/01/1982,03,...,330.0,03/27/1991,03/27/1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111143,2025Q3,KY0842839,2037882,NaN,01/01/1979,12/31/1981,01/01/1979,12/31/1981,10/01/1997,03,...,330.0,07/10/1990,03/27/1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87592,2025Q3,KY1170995,5631289,NaN,01/01/1981,12/31/1981,01/01/1981,12/31/1981,NaN,03,...,330.0,07/10/1990,03/27/1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
violations = violations.rename(columns={
    "NON_COMPL_PER_BEGIN_DATE": "noncompl_begin",
    "NON_COMPL_PER_END_DATE": "noncompl_end",
    "VIOLATION_CATEGORY_CODE": "violation_category",
    "IS_HEALTH_BASED_IND": "is_health_based",
    "CONTAMINANT_CODE": "contaminant_code",
    "PWSID": "pwsid"
})
violations.columns


Index(['SUBMISSIONYEARQUARTER', 'pwsid', 'VIOLATION_ID', 'FACILITY_ID',
       'COMPL_PER_BEGIN_DATE', 'COMPL_PER_END_DATE', 'noncompl_begin',
       'noncompl_end', 'PWS_DEACTIVATION_DATE', 'VIOLATION_CODE',
       'violation_category', 'is_health_based', 'contaminant_code',
       'VIOL_MEASURE', 'UNIT_OF_MEASURE', 'FEDERAL_MCL', 'STATE_MCL',
       'IS_MAJOR_VIOL_IND', 'SEVERITY_IND_CNT', 'CALCULATED_RTC_DATE',
       'VIOLATION_STATUS', 'PUBLIC_NOTIFICATION_TIER',
       'CALCULATED_PUB_NOTIF_TIER', 'VIOL_ORIGINATOR_CODE', 'SAMPLE_RESULT_ID',
       'CORRECTIVE_ACTION_ID', 'RULE_CODE', 'RULE_GROUP_CODE',
       'RULE_FAMILY_CODE', 'VIOL_FIRST_REPORTED_DATE',
       'VIOL_LAST_REPORTED_DATE', 'ENFORCEMENT_ID', 'ENFORCEMENT_DATE',
       'ENFORCEMENT_ACTION_TYPE_CODE', 'ENF_ACTION_CATEGORY',
       'ENF_ORIGINATOR_CODE', 'ENF_FIRST_REPORTED_DATE',
       'ENF_LAST_REPORTED_DATE'],
      dtype='object')

In [35]:
violations["noncompl_begin"] = pd.to_datetime(violations["noncompl_begin"], errors="coerce")
violations["noncompl_end"] = pd.to_datetime(violations["noncompl_end"], errors="coerce")


## 6. Kentucky-Only Verification

Because the SDWA violation dataset can include records from other states, I used the PWSID and STATE_CODE to make sure the data only has Kentucky violations.


In [36]:
systems["STATE_CODE"].value_counts(dropna=False).head(10)


STATE_CODE
KY    2214
Name: count, dtype: int64

In [37]:
check = violations.merge(
    systems[["PWSID", "STATE_CODE"]],
    left_on="pwsid",
    right_on="PWSID",
    how="left"
)

check["STATE_CODE"].value_counts(dropna=False).head(20)


STATE_CODE
KY    136596
Name: count, dtype: int64

## Week 1 Summary

In this first part of the Keystone Project, I focused on getting the SDWA data into a workable state so I could see what I was actually dealing with.

One of the first problems I ran into was the size of the SDWA violation files. I tried to use Git Large File Storage (LFS) so the full dataset could be tracked in the repository, but could not get it push. So, I changed my approach.

I created a Kentucky only data extraction from the full SDWA files. This allowed me to keep the complete raw data locally while committing a smaller, focused subset of the data that is directly relevant to this project. This extraction step also made the dataset more manageable and aligned better with the scope of the Keystone Project. If this is not allowed or frowned upon, please let me know. 

After I got the Kentucky only datasets loaded, I looked at key variables, missing data and checked for duplicates. I performed basic cleaning and wrangling steps. I also made sure that all violation records were for Kentucky public water systems. 

# Keystone Project – Week 2  
## Exploratory Data Analysis (EDA) and Early Visualizations

In this step of my Keystone Project, I will focus on EDA. Using the Kentucky only dataset extracted from the Kentucky Safe Drinking Water Act (SDWA) violations data, I hope to see some kind of relationship or pattern and maybe get some answers on what we are drinking.


## Data Prep for EDA

Before continuing, I needed to confirm data types, check for duplicates, and make sure all cleaning steps from week 1 worked correctly. I reloaded the data and some of previous code used in Week 1 so this section can be ran separate from Week 1 if preferred. 


In [10]:
from pathlib import Path
import pandas as pd

project_root = Path("..")
data_folder = project_root / "datasets" / "SDWA_ky"

violations = pd.read_csv(data_folder / "ky_violations_enforcement.csv", low_memory=False)
systems = pd.read_csv(data_folder / "ky_pub_water_systems.csv", low_memory=False)
codes = pd.read_csv(data_folder / "ref_contaminant_codes.csv", low_memory=False)


In [11]:
violations = violations.rename(columns={
    "NON_COMPL_PER_BEGIN_DATE": "noncompl_begin",
    "NON_COMPL_PER_END_DATE": "noncompl_end",
    "VIOLATION_CATEGORY_CODE": "violation_category",
    "IS_HEALTH_BASED_IND": "is_health_based",
    "CONTAMINANT_CODE": "contaminant_code",
    "PWSID": "pwsid"
})
violations["noncompl_begin"] = pd.to_datetime(violations["noncompl_begin"], errors="coerce")
violations["noncompl_end"] = pd.to_datetime(violations["noncompl_end"], errors="coerce")

In [12]:
violations.shape
systems.shape
codes.shape


(871, 3)

In [13]:
violations.duplicated().sum()


np.int64(0)

In [14]:
violations[["noncompl_begin", "noncompl_end"]].dtypes


noncompl_begin    datetime64[ns]
noncompl_end      datetime64[ns]
dtype: object

I created a subset of health-based violations to explore which violations may pose direct risks to public health.  
This subset will be useful later when comparing violation types and identifying patterns.


In [15]:
health_violations = violations[violations["is_health_based"] == "Y"]
health_violations.shape


(14764, 38)

In [16]:
violations.sort_values("noncompl_begin").head()


,SUBMISSIONYEARQUARTER,pwsid,VIOLATION_ID,FACILITY_ID,COMPL_PER_BEGIN_DATE,COMPL_PER_END_DATE,noncompl_begin,noncompl_end,PWS_DEACTIVATION_DATE,VIOLATION_CODE,...,RULE_FAMILY_CODE,VIOL_FIRST_REPORTED_DATE,VIOL_LAST_REPORTED_DATE,ENFORCEMENT_ID,ENFORCEMENT_DATE,ENFORCEMENT_ACTION_TYPE_CODE,ENF_ACTION_CATEGORY,ENF_ORIGINATOR_CODE,ENF_FIRST_REPORTED_DATE,ENF_LAST_REPORTED_DATE
106626,2025Q3,VA1105073,1815181,NaN,06/01/1977,05/31/1981,1977-06-01,1981-05-31,11/01/1992,03,...,340.0,09/30/1981,09/30/1981,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64879,2025Q3,VA1105073,1815681,NaN,06/01/1977,05/31/1981,1977-06-01,1981-05-31,11/01/1992,03,...,340.0,09/30/1981,09/30/1981,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100384,2025Q3,KY1022558,1049681,NaN,01/01/1978,12/31/1980,1978-01-01,1980-12-31,07/01/1982,03,...,330.0,03/27/1991,03/27/1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72112,2025Q3,KY0392416,988881,NaN,01/01/1978,12/31/1980,1978-01-01,1980-12-31,10/01/1981,03,...,330.0,03/27/1991,03/27/1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64674,2025Q3,KY0672475,1017281,NaN,01/01/1978,12/31/1980,1978-01-01,1980-12-31,08/01/2003,03,...,330.0,07/10/1990,03/27/1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Sorting violations by the initial noncompliance can tell us how long some violations have been ongoing.  



## EDA Question 1: What types of SDWA violations are most common in Kentucky?

To begin exploring the data, I wanted to understand which violations occur most frequently across Kentucky public water systems.


In [17]:
violation_counts = (
    violations
    .groupby("violation_category")
    .size()
    .sort_values(ascending=False)
)

violation_counts


violation_category
MR       94371
MCL      11306
Other     5892
TT        3456
MON        810
RPT         97
MRDL         2
dtype: int64

### Observation


Monitoring and reporting appear far more frequently than other SDWA violation category in Kentucky. The most common category, **MR**, represents *Monitoring and Reporting* violations based on EPA SDWA classification codes.

This stood out because it suggests that many of the violations in the dataset are related to compliance and paperwork requirements rather than direct contaminant exceedances. While these types of violations don’t always mean the water itself is unsafe, they still matter because they affect how water systems are monitored and how potential risks are identified and addressed over time.
